In [ ]:
!pip install tensorflow_addons
# !pip install keras
!pip3 install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 32.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 27.6 MB/s 


In [ ]:
from keras.layers import Bidirectional, Dense, Embedding, Input, Lambda, LSTM, RepeatVector, TimeDistributed, Layer, Activation, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.layers.advanced_activations import ELU
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras import backend as K
from keras.models import Model
# from scipy import spatial
import tensorflow as tf
# import pandas as pd
import numpy as np
# import codecs
import pickle5 as pk
import os
from pathlib import Path
import tensorflow_addons as tfa

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from tensorflow.python.framework.ops import disable_eager_execution 
disable_eager_execution()

In [ ]:
path_to_training_data = Path.cwd().joinpath("quantum_material_data.pkl")
with open(path_to_training_data, 'rb') as f:
    data = pk.load(f)


dataset = data["training_data"]
tokenizer = data["tokenizer"]


number_of_equations = dataset.shape[0]

In [ ]:
dataset.shape

(8464, 200)

In [ ]:
number_of_equations

8464

In [ ]:
np.random.shuffle(dataset)

In [ ]:
dataset

array([[ 2,  2,  6, ...,  0,  0,  0],
       [ 6,  2,  2, ...,  0,  0,  0],
       [25, 27,  1, ...,  0,  0,  0],
       ...,
       [30, 11, 30, ...,  0,  0,  0],
       [ 2,  4,  8, ...,  0,  0,  0],
       [ 1,  3,  1, ...,  0,  0,  0]], dtype=int32)

In [ ]:
training = dataset[:7400].astype(np.int32)
test = dataset[7400:8400].astype(np.int32)

In [ ]:
#used to be 25
batch_size = 25
epochs = 100
max_len = len(dataset[1])
latent_dim = 100
intermediate_dim = 50
emb_dim = 100
epsilon_std = 1.0
kl_weight = 0.1
number_of_letters = len(tokenizer.word_index) + 1
learning_rate = 1e-5
num_sampled=500
act = ELU()

In [ ]:
x = Input(batch_shape=(None, max_len))
x_embed = Embedding(max_len, emb_dim,
                            input_length=max_len, trainable=False)(x)
h = Bidirectional(LSTM(intermediate_dim, return_sequences=False, recurrent_dropout=0.5), merge_mode='concat')(x_embed)
#h = Bidirectional(LSTM(intermediate_dim, return_sequences=False), merge_mode='concat')(h)
h = Dropout(0.5)(h)
h = Dense(intermediate_dim, activation='linear')(h)
h = act(h)
h = Dropout(0.5)(h)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# we instantiate these layers separately so as to reuse them later
repeated_context = RepeatVector(max_len)
decoder_h = LSTM(intermediate_dim, return_sequences=True, recurrent_dropout=0.5)
decoder_mean = TimeDistributed(Dense(number_of_letters, activation='linear'))#softmax is applied in the seq2seqloss by tf
h_decoded = decoder_h(repeated_context(z))
x_decoded_mean = decoder_mean(h_decoded)

In [ ]:


# placeholder loss
def zero_loss(y_true, y_pred):
    return K.zeros_like(y_pred)

# #=========================== Necessary only if you want to use Sampled Softmax =======================#
# #Sampled softmax
# logits = tf.constant(np.random.randn(batch_size, max_len, NB_WORDS), tf.float32)
# targets = tf.constant(np.random.randint(NB_WORDS, size=(batch_size, max_len)), tf.int32)
# proj_w = tf.constant(np.random.randn(NB_WORDS, NB_WORDS), tf.float32)
# proj_b = tf.constant(np.zeros(NB_WORDS), tf.float32)

# def _sampled_loss(labels, logits):
#     labels = tf.cast(labels, tf.int64)
#     labels = tf.reshape(labels, [-1, 1])
#     logits = tf.cast(logits, tf.float32)
#     return tf.cast(
#                     tf.nn.sampled_softmax_loss(
#                         proj_w,
#                         proj_b,
#                         labels,
#                         logits,
#                         num_sampled=num_sampled,
#                         num_classes=NB_WORDS),
#                     tf.float32)
# softmax_loss_f = _sampled_loss
#====================================================================================================#


In [ ]:
# Custom VAE loss layer
class CustomVariationalLayer(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)
        self.target_weights = tf.constant(np.ones((batch_size, max_len)), tf.float32)

    def vae_loss(self, x, x_decoded_mean):
        #xent_loss = K.sum(metrics.categorical_crossentropy(x, x_decoded_mean), axis=-1)
        labels = tf.cast(x, tf.int32)
        xent_loss = K.sum(tfa.seq2seq.sequence_loss(x_decoded_mean, labels, 
                                                     weights=self.target_weights,
                                                     average_across_timesteps=False,
                                                     average_across_batch=False), axis=-1)
                                                     #softmax_loss_function=softmax_loss_f), axis=-1)#, uncomment for sampled doftmax
        kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        x_decoded_mean = inputs[1]
        print(x.shape, x_decoded_mean.shape)
        loss = self.vae_loss(x, x_decoded_mean)
        self.add_loss(loss, inputs=inputs)
        # we don't use this output, but it has to have the correct shape:
        return K.ones_like(x)

loss_layer = CustomVariationalLayer()([x, x_decoded_mean])
vae = Model(x, [loss_layer])
opt = Adam(lr=0.01) #SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
vae.compile(optimizer='adam', loss=[zero_loss])
vae.summary()

(None, 200) (25, 200, 59)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 200)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 200, 100)     20000       ['input_1[0][0]']                
                                                                                                  
 bidirectional (Bidirectional)  (None, 100)          60400       ['embedding[0][0]']              
                                                                                                  
 dropout (Dropout)              (None, 100)          0           ['bidirectional[0][0]']          
                                                                    

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
def create_model_checkpoint(dir, model_name):
    filepath = dir + '/' + model_name + ".h5" #-{epoch:02d}-{decoded_mean:.2f}
    directory = os.path.dirname(filepath)
    try:
        os.stat(directory)
    except:
        os.mkdir(directory)
    checkpointer = ModelCheckpoint(filepath=filepath, verbose=1, save_best_only=False)
    return checkpointer

checkpointer = create_model_checkpoint('models', 'vae_seq2seq')

nb_epoch=100

vae.fit(training, training,
      epochs=epochs,
      batch_size=batch_size,
      validation_data=(test, test), callbacks=[checkpointer])
# n_steps = (800000/2)/batch_size #we use the first 800000
# for counter in range(nb_epoch):
#     print('-------epoch: ',counter,'--------')
#     vae.fit_generator(sent_generator(TRAIN_DATA_FILE, batch_size/2),
#                           steps_per_epoch=n_steps, epochs=1, callbacks=[checkpointer],
#                           validation_data=(data_1_val, data_1_val))
    
vae.save('models/vae_lstm800k32dim96hid.h5')

Train on 7400 samples, validate on 1000 samples
Epoch 1/100
1375/7400 [====>.........................] - ETA: 5:08 - loss: 497.1759

In [ ]:
encoder = Model(x, z_mean)

# build a generator that can sample sentences from the learned distribution
decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h(repeated_context(decoder_input))
_x_decoded_mean = decoder_mean(_h_decoded)
_x_decoded_mean = Activation('softmax')(_x_decoded_mean)
generator = Model(decoder_input, _x_decoded_mean)

In [ ]:
index2word = {v: k for k, v in word_index.items()}
sent_encoded = encoder.predict(data_1_val, batch_size = 16)
x_test_reconstructed = generator.predict(sent_encoded)
                                         
sent_idx = 672
reconstructed_indexes = np.apply_along_axis(np.argmax, 1, x_test_reconstructed[sent_idx])
#np.apply_along_axis(np.max, 1, x_test_reconstructed[sent_idx])
#np.max(np.apply_along_axis(np.max, 1, x_test_reconstructed[sent_idx]))
word_list = list(np.vectorize(index2word.get)(reconstructed_indexes))
word_list
original_sent = list(np.vectorize(index2word.get)(data_1_val[sent_idx]))
original_sent

In [ ]:
# function to parse a sentence
def sent_parse(sentence, mat_shape):
    sequence = tokenizer.texts_to_sequences(sentence)
    padded_sent = pad_sequences(sequence, maxlen=MAX_SEQUENCE_LENGTH)
    return padded_sent#[padded_sent, sent_one_hot]

# input: encoded sentence vector
# output: encoded sentence vector in dataset with highest cosine similarity
def find_similar_encoding(sent_vect):
    all_cosine = []
    for sent in sent_encoded:
        result = 1 - spatial.distance.cosine(sent_vect, sent)
        all_cosine.append(result)
    data_array = np.array(all_cosine)
    maximum = data_array.argsort()[-3:][::-1][1]
    new_vec = sent_encoded[maximum]
    return new_vec

# input: two points, integer n
# output: n equidistant points on the line between the input points (inclusive)
def shortest_homology(point_one, point_two, num):
    dist_vec = point_two - point_one
    sample = np.linspace(0, 1, num, endpoint = True)
    hom_sample = []
    for s in sample:
        hom_sample.append(point_one + s * dist_vec)
    return hom_sample

# input: original dimension sentence vector
# output: sentence text
def print_latent_sentence(sent_vect):
    sent_vect = np.reshape(sent_vect,[1,latent_dim])
    sent_reconstructed = generator.predict(sent_vect)
    sent_reconstructed = np.reshape(sent_reconstructed,[max_len,NB_WORDS])
    reconstructed_indexes = np.apply_along_axis(np.argmax, 1, sent_reconstructed)
    np.apply_along_axis(np.max, 1, x_test_reconstructed[sent_idx])
    np.max(np.apply_along_axis(np.max, 1, x_test_reconstructed[sent_idx]))
    word_list = list(np.vectorize(index2word.get)(reconstructed_indexes))
    w_list = [w for w in word_list if w]
    print(' '.join(w_list))
    #print(word_list)
        
def new_sents_interp(sent1, sent2, n):
    tok_sent1 = sent_parse(sent1, [15])
    tok_sent2 = sent_parse(sent2, [15])
    enc_sent1 = encoder.predict(tok_sent1, batch_size = 16)
    enc_sent2 = encoder.predict(tok_sent2, batch_size = 16)
    test_hom = shortest_homology(enc_sent1, enc_sent2, n)
    for point in test_hom:
        print_latent_sentence(point)

In [ ]:
sentence1=['where can i find a book on machine learning']
mysent = sent_parse(sentence1, [15])
mysent_encoded = encoder.predict(mysent, batch_size = 16)
print_latent_sentence(mysent_encoded)
print_latent_sentence(find_similar_encoding(mysent_encoded))

sentence2=['how can i become a successful entrepreneur']
mysent2 = sent_parse(sentence2, [15])
mysent_encoded2 = encoder.predict(mysent2, batch_size = 16)
print_latent_sentence(mysent_encoded2)
print_latent_sentence(find_similar_encoding(mysent_encoded2))
print('-----------------')

new_sents_interp(sentence1, sentence2, 6)